# Blackjack(MDP)

En esta libreta se intenta resolver el problema de jugar blackjack(21) usando MDP.

Reglas:
* El mazo es infinito, en cualquier estado la probabilidad de salir una cartas es la misma.
* Se gana cuando la suma de las cartas es 21, se tengan 4 cartas sin pasarse de 21, el dealer se pase de 21 y el jugador no o cuando la suma del jugador es mayor a la del dealer.
* El jugador siempre empieza jugando y el diler juega una vez que el jugador pase pase con una suma menor a 21.
* El dealer pide carta hasta tener una suma mayor a 16.

Representacion:

$ S = (sj,sd,A,cj)$

$ a = \{ pedir, quedarse \} $

$ S_f =  \{ (0,0,0,1), (0,0,0,-1) \} $

$ \rho =  probabilidad de pasar de un estado a otro$

$ r = $ 1 si gana, -1 si pierde y 0 en otro caso

### Acciones validas

In [1]:
a = ["pedir", "quedarse"]


2-element Array{String,1}:
 "pedir"   
 "quedarse"

### Conjunto de estados
Primero definimos los estados posibles.

Representacion de un estado:

S=(sj,sd,A,cj)

donde:

* sj=suma de las cartas del jugador
* sd=suma de las cartas del dealer
* A=indica si el jugador tiene almenos un As
* cj=indica cuantas cartas tiene el jugador

Mis ideas para representar los estado:
* Cuando tengo un As lo considero como 11 de ser posible. Por ejemplo si tengo 2 y A considero que tengo 13, por lo cual el valor minimo de sj en mi representacion es 4(2+2).
* El estado [0,0,0,1] en mi representa todos los estados ganadores tales como {[21, 5, 0, 2], [21, 6, 0, 2],...}
* El estado [0,0,0,-1] en mi representa todos los estados perdedores tales como {[21, 5, 0, 2], [21, 6, 0, 2],...}
* Estos 2 ultimos puntos los agregue por que creo que no es necesario definir todos estos estados finales ya que se pueden considerar de estas dos maneras y con una funcion calcular la probabilidad de pasar de un estado por ejemplo [17,10,0,3] a un estado ganador realizando la accion "pedir"([18,10,0,4],[19,10,0,4],[20,10,0,4],[21,10,0,4]), se sumarian las probabilidades y se multiplicaria por la recompenza del estado [0,0,0,1] que en este caso es 1.
* El unico estado terminal fuera del conjunto [0,0,0,1] seria [21,sd,1,2] que seria cuando con las primeras 2 cartas saca 21.

Nota: Son solo ideas.

In [2]:
s = [[0,0,0,1],[0,0,0,-1]] #estados ganadores y perdedores
for d in 2:11
    for i in 4:20
        push!(s, [i, d, 0, 2]) #estados donde el jugador tiene 2 cartas sin As
    end
    for i in 12:21
        push!(s, [i, d, 1, 2]) #estados donde el jugador tiene 2 cartas con almenos un As
    end
    for i in 6:20
        push!(s, [i, d, 0, 3]) #estados donde el jugador tiene 3 cartas sin As
    end
    for i in 13:20
        push!(s, [i, d, 1, 3]) #estados donde el jugador tiene 3 cartas con almenos un As
    end
end
s

502-element Array{Array{Int64,1},1}:
 [0, 0, 0, 1]  
 [0, 0, 0, -1] 
 [4, 2, 0, 2]  
 [5, 2, 0, 2]  
 [6, 2, 0, 2]  
 [7, 2, 0, 2]  
 [8, 2, 0, 2]  
 [9, 2, 0, 2]  
 [10, 2, 0, 2] 
 [11, 2, 0, 2] 
 [12, 2, 0, 2] 
 [13, 2, 0, 2] 
 [14, 2, 0, 2] 
 ⋮             
 [17, 11, 0, 3]
 [18, 11, 0, 3]
 [19, 11, 0, 3]
 [20, 11, 0, 3]
 [13, 11, 1, 3]
 [14, 11, 1, 3]
 [15, 11, 1, 3]
 [16, 11, 1, 3]
 [17, 11, 1, 3]
 [18, 11, 1, 3]
 [19, 11, 1, 3]
 [20, 11, 1, 3]

### Estructura
* s: los estados
* sf: los estados finales
* a: acciones legales
* p: funcion de probabilidad
* r: funcion de recompenza

In [3]:
struct MDP
    s
    sf
    a
    ρ
    r
end

In [4]:
function iter_value(mdp, r)
    v = Dict(s => 0.0 for s in mdp.s)
    v_p = Dict(s => 0.0 for s in mdp.s)

    salir = false
    
    while !salir
        for s in keys(v)
            if s in mdp.sf
                v_p[s] = mdp.r(s, nothing, s)
            else
                v_p[s] = maximum([sum([mdp.ρ(s, a, s_) * (mdp.r(s, a, s_) + r * v[s_]) 
                             for s_ in mdp.s]) for a in mdp.a])
            end
            salir = true
                            
            for s in keys(v)
                if v_p[s] > v[s]
                    salir = false
                    v[s] = v_p[s]
                end
            end
            
            if salir
                break
            end
        end
    end
    
    pπ = Dict(s => "" for s in mdp.s)
    for s in keys(v)

        temp = Dict(a => sum([mdp.ρ(s, a, s_) * v[s_] for s_ in mdp.s])
                            for a in mdp.a)

        pπ[s] = findmax(temp)[2]
    end
    return pπ
end

iter_value (generic function with 1 method)

In [5]:
function ρ(s, a, s_)

    if a == "quedarse"
        if s == s_
            return 1
        else
            return 0
        end
    elseif (s_==[0,0,0,1])
        d = 21 - s[1]
        if s[4]==3
            if d>=10
               return 1
            else
               return  1/13 * d
            end  
        else
            if d>10
               return 1
            elseif(d==10)
               return 4/13
            else
               return 1
            end   
        end
    elseif (s_==[0,0,0,-1])
        d = 21 - s[1]
        if d>10
           return 0
        else
           return  4/13+1/13*(10-d)
        end    
    elseif (s_[4] == s[4] + 1 && s_[1]>s[1])
        d = s_[1] - s[1]
        if  (d >= 1 && d <= 9) || d==11 
            return 1/13
        elseif(d==10)
            return 4/13
        else
            return 1
        end
    end
    
    return 0
end

ρ (generic function with 1 method)

In [6]:
function r(s, a, s_)
    if s_ == [0,0,0,1] 
        return 1
    elseif s_ == [0,0,0,-1] 
        return -1
    else
        return 0
    end
end

r (generic function with 1 method)

In [7]:
blackJack = MDP(s, [[0,0,0,1],[0,0,0,-1]], a, ρ, r)
rw = 0.5

0.5

In [8]:
pπ = iter_value(blackJack, rw)

Dict{Array{Int64,1},String} with 502 entries:
  [13, 9, 1, 3]  => "quedarse"
  [11, 6, 0, 2]  => "pedir"
  [14, 8, 1, 3]  => "quedarse"
  [14, 2, 1, 3]  => "quedarse"
  [17, 6, 0, 2]  => "quedarse"
  [7, 11, 0, 2]  => "pedir"
  [14, 10, 1, 3] => "quedarse"
  [18, 4, 0, 3]  => "quedarse"
  [18, 6, 0, 2]  => "quedarse"
  [8, 10, 0, 3]  => "quedarse"
  [14, 3, 1, 3]  => "quedarse"
  [20, 4, 0, 2]  => "quedarse"
  [15, 2, 0, 3]  => "quedarse"
  [16, 8, 1, 3]  => "quedarse"
  [17, 8, 1, 3]  => "quedarse"
  [20, 5, 0, 2]  => "quedarse"
  [17, 6, 1, 2]  => "quedarse"
  [14, 4, 1, 2]  => "quedarse"
  [8, 7, 0, 3]   => "quedarse"
  [19, 6, 0, 2]  => "quedarse"
  [20, 9, 1, 3]  => "quedarse"
  [5, 5, 0, 2]   => "pedir"
  [17, 7, 1, 3]  => "quedarse"
  [17, 7, 0, 2]  => "quedarse"
  [13, 8, 1, 2]  => "quedarse"
  ⋮              => ⋮

In [9]:
for s in s
    println(s, pπ[s])
end

[0, 0, 0, 1]pedir
[0, 0, 0, -1]quedarse
[4, 2, 0, 2]pedir
[5, 2, 0, 2]pedir
[6, 2, 0, 2]pedir
[7, 2, 0, 2]pedir
[8, 2, 0, 2]pedir
[9, 2, 0, 2]pedir
[10, 2, 0, 2]pedir
[11, 2, 0, 2]pedir
[12, 2, 0, 2]pedir
[13, 2, 0, 2]quedarse
[14, 2, 0, 2]quedarse
[15, 2, 0, 2]quedarse
[16, 2, 0, 2]quedarse
[17, 2, 0, 2]quedarse
[18, 2, 0, 2]quedarse
[19, 2, 0, 2]quedarse
[20, 2, 0, 2]quedarse
[12, 2, 1, 2]pedir
[13, 2, 1, 2]quedarse
[14, 2, 1, 2]quedarse
[15, 2, 1, 2]quedarse
[16, 2, 1, 2]quedarse
[17, 2, 1, 2]quedarse
[18, 2, 1, 2]quedarse
[19, 2, 1, 2]quedarse
[20, 2, 1, 2]quedarse
[21, 2, 1, 2]quedarse
[6, 2, 0, 3]quedarse
[7, 2, 0, 3]quedarse
[8, 2, 0, 3]quedarse
[9, 2, 0, 3]quedarse
[10, 2, 0, 3]quedarse
[11, 2, 0, 3]quedarse
[12, 2, 0, 3]quedarse
[13, 2, 0, 3]quedarse
[14, 2, 0, 3]quedarse
[15, 2, 0, 3]quedarse
[16, 2, 0, 3]quedarse
[17, 2, 0, 3]quedarse
[18, 2, 0, 3]quedarse
[19, 2, 0, 3]quedarse
[20, 2, 0, 3]quedarse
[13, 2, 1, 3]quedarse
[14, 2, 1, 3]quedarse
[15, 2, 1, 3]quedarse
[16, 2, 1,

[15, 11, 0, 2]quedarse
[16, 11, 0, 2]quedarse
[17, 11, 0, 2]quedarse
[18, 11, 0, 2]quedarse
[19, 11, 0, 2]quedarse
[20, 11, 0, 2]quedarse
[12, 11, 1, 2]pedir
[13, 11, 1, 2]quedarse
[14, 11, 1, 2]quedarse
[15, 11, 1, 2]quedarse
[16, 11, 1, 2]quedarse
[17, 11, 1, 2]quedarse
[18, 11, 1, 2]quedarse
[19, 11, 1, 2]quedarse
[20, 11, 1, 2]quedarse
[21, 11, 1, 2]quedarse
[6, 11, 0, 3]quedarse
[7, 11, 0, 3]quedarse
[8, 11, 0, 3]quedarse
[9, 11, 0, 3]quedarse
[10, 11, 0, 3]quedarse
[11, 11, 0, 3]quedarse
[12, 11, 0, 3]quedarse
[13, 11, 0, 3]quedarse
[14, 11, 0, 3]quedarse
[15, 11, 0, 3]quedarse
[16, 11, 0, 3]quedarse
[17, 11, 0, 3]quedarse
[18, 11, 0, 3]quedarse
[19, 11, 0, 3]quedarse
[20, 11, 0, 3]quedarse
[13, 11, 1, 3]quedarse
[14, 11, 1, 3]quedarse
[15, 11, 1, 3]quedarse
[16, 11, 1, 3]quedarse
[17, 11, 1, 3]quedarse
[18, 11, 1, 3]quedarse
[19, 11, 1, 3]quedarse
[20, 11, 1, 3]quedarse
